# Concept
- FOLD別のアンサンブルを行う。

# Import

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import random
import warnings
import glob
import shutil
import tensorflow as tf
import tensorflow.keras.applications.efficientnet as efn
import tensorflow.keras.backend as K
from pathlib import Path
from tensorflow.random import set_seed
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, concatenate, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

# Settings

In [ ]:
CFG = {
    'load_dir':"/kaggle/input/ranzcr-clip-catheter-line-classification/",
    'train_csv_name':"train.csv",
    'submit_csv_name':"sample_submission.csv",
    'batch_size':32,
    'epoch':20,
    'seed':20210222,
    'size_x':(224, 240, 260, 300, 380, 456, 528, 600),  # efficientnetB0～B07のデフォルト
    'size_y':(224, 240, 260, 300, 380, 456, 528, 600),  # efficientnetB0～B07のデフォルト
    'channel':3,
    'fold':3,
    'label_num':11,
}

# Functions

In [ ]:
def seed_set(seed):
    """
    Seed値の固定
    """
    # tendorflow seed
    set_seed(seed)
    # for numpy.random
    np.random.seed(seed)
    # for built-in random
    random.seed(seed)
    # for hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)

def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    """
    ジェネレータ内で画像パスから読み込んで前処理する
    """
    def decode(path):
        
        file_bytes = tf.io.read_file(path) 

        if ext == 'png':
            # PNGでエンコードされた画像を uint8 または uint16 のテンソルにデコード
            img = tf.image.decode_png(file_bytes, channels=3) 
        elif ext in ['jpg', 'jpeg']:
            # JPEGでエンコードされた画像をuint8テンソルにデコード
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        # float32型に変換して正規化
        img = tf.cast(img, tf.float32) / 255.0
        # リサイズ処理
        img = tf.image.resize(img, target_size)
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

# データ増強
def build_augmenter(with_labels=True):
    """
    ジェネレータ内でデータ増強関数
    """
    def augment(img):
        img = tf.image.random_flip_left_right(img)  # ランダムで水平方向に反転
        img = tf.image.random_flip_up_down(img)  # ランダムで垂直方向に反転
#         img = tf.image.random_saturation(img, 0.8, 1.2)  # RGB画像の彩度を調整
        img = tf.image.random_brightness(img, 0.2) # 画像の明るさを調整
        img = tf.image.random_contrast(img, 0.8, 1.2)  # 画像のコントラストを調整
#         img = tf.image.random_hue(img, 0.2)  # RGB画像の色相を調整
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    """
    パイプラインを構築してデータを吐き出すジェネレータを作成する関数。
    """
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    # データセット作成
    dset = tf.data.Dataset.from_tensor_slices(slices)
    # 値を直接変えるパイプライン(※画像の前処理(リサイズ・正規化 etc)を行う)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)  
    # 高速化のため前処理後のデータをキャッシュして保持しておく。
    dset = dset.cache(cache_dir) if cache else dset  
    # 値を直接変えるパイプライン(データ増強)
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset 
    # データを指定回数Repeatする 
    dset = dset.repeat() if repeat else dset
    # データをshuffleする
    dset = dset.shuffle(shuffle) if shuffle else dset
    # モデルの訓練中にバックグラウンドでデータセットバッチ取得。
    dset = dset.batch(bsize).prefetch(AUTO)
    return dset

def load_model(h_size_x, h_size_y, h_channel, h_n_labels, h_trainable=True):
    """
    imagenet学習済のEfficientNetを構築する。
    """
    model_path = '../input/tfkeras-efficientnet-weights/efficientnetb2_notop.h5'  # imagenet
    # EfficientNetのinagenet学習済モデルを読み込み
    eff_net = efn.EfficientNetB2(weights=model_path, include_top=False, input_shape=(h_size_x, h_size_y, h_channel))
    # Trueの場合、EfficientNetのに対してFineチューニングを実施する。
    eff_net.trainable = h_trainable
    x = eff_net.output
    x1 = GlobalAveragePooling2D()(x) 
    x2 = GlobalMaxPooling2D()(x)
    x = concatenate([x1, x2])
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(252, activation='relu')(x)
    x = Dropout(0.5)(x)
    # マルチラベルのため、出力層はsofmaxではなく、sigmoid関数を設定する。
    predictions = Dense(h_n_labels, activation='sigmoid')(x)
    model = Model(inputs = eff_net.input, outputs = predictions)
    return model

def load_dataset(h_train_paths, h_labels, h_size_x, h_size_y, h_n_splits, h_seed=None):
    """
    データセットgneratorを取得する
    """
    # 学習データの前処置の定義
    decoder = build_decoder(with_labels=True, target_size=(h_size_x, h_size_y))

    dtrain_list = []
    dvalid_list = []
    train_paths_len_list = []

    # Ffold
#     kf = StratifiedKFold(h_n_splits, shuffle=True)
    kf = KFold(n_splits=h_n_splits, shuffle=True)
    for train_index, test_index in kf.split(h_train_paths, h_labels):
        train_data = h_train_paths[train_index]
        train_label = h_labels[train_index]
        val_data = h_train_paths[test_index]
        val_label = h_labels[test_index]

        train_paths_len_list.append(len(train_data))

        # trainデータgeneratorを生成
        dtrain_list.append(build_dataset(
            train_data, train_label, bsize=CFG['batch_size'], augment=True,
            cache_dir='./cache/tf_cache', decode_fn=decoder
        ))

        # validtionデータgeneratorを生成
        dvalid_list.append(build_dataset(
            val_data, val_label, bsize=CFG['batch_size'], 
            repeat=False, shuffle=False, augment=True, 
            cache_dir='./cache/tf_cache', decode_fn=decoder
        ))

    return dtrain_list, dvalid_list, train_paths_len_list

# 各種パス取得とラベル生成

In [ ]:
# データの読み込み
load_dir = CFG['load_dir']

# trainデータのパスを読み込み
df = pd.read_csv(CFG['load_dir'] + CFG['train_csv_name'])
train_paths = CFG['load_dir'] + "train/" + df['StudyInstanceUID'] + '.jpg'

# testデータのパスを読み込み
sub_df = pd.read_csv(CFG['load_dir'] + CFG['submit_csv_name'])
test_paths = CFG['load_dir'] + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

# labelデータを生成
label_cols = sub_df.columns[1:]
labels = df[label_cols].values

# 学習

In [ ]:
# seed値を固定する
seed_set(CFG["seed"])

# inputのsizeを設定
size_x = CFG["size_x"][2]
size_y = CFG["size_y"][2]

# データセットをロードする
dtrain_list, dvalid_list, train_len_list = load_dataset(train_paths, labels, size_x, size_y, CFG['fold'], h_seed=CFG["seed"])

# Fold
for i, (dtrain, dvalid, shape) in enumerate(zip(dtrain_list, dvalid_list, train_len_list)):
    # １エポックで全画像を処置する
    steps_per_epoch = shape // CFG['batch_size']
    # imagenet学習済のモデルを取得
    model = load_model(size_x, size_y, CFG["channel"], CFG["label_num"], h_trainable=True)
    # モデルコンパイル
    model.compile(
        optimizer=optimizers.Adam(lr=0.001),
        loss='binary_crossentropy', 
        metrics=[tf.keras.metrics.AUC(multi_label=True)]
    )
    # checkpointファイルを生成するコールバック
    checkpoint = tf.keras.callbacks.ModelCheckpoint('./model_Fold{}.h5'.format(str(i)), save_best_only=True, monitor='val_loss', mode='min')

    # 学習が停滞したら、学習率を半減するコールバック
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3, min_lr=1e-6, mode='min')
    
    # 学習
    history = model.fit(
        dtrain, 
        epochs=CFG["epoch"],
        verbose=1,
        callbacks=[checkpoint, lr_reducer],
        steps_per_epoch=steps_per_epoch,
        validation_data=dvalid
        )

# 推論

In [ ]:
# モデル構造の読み込み
size_x = CFG["size_x"][2]
size_y = CFG["size_y"][2]
model = load_model(size_x, size_y, CFG["channel"], CFG["label_num"], h_trainable=True)

# testデータのパスを読み込み
sub_df = pd.read_csv(CFG['load_dir'] + CFG['submit_csv_name'])
test_paths = CFG['load_dir'] + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

# テストデータの前処置の定義
test_decoder = build_decoder(with_labels=False, target_size=(size_x, size_y))

# testデータgeneratorを生成
dtest = build_dataset(
    test_paths, bsize=CFG['batch_size'], repeat=False, 
    shuffle=False, augment=False, cache=False, 
    decode_fn=test_decoder
)

# 推論
for i in range(CFG['fold']):
    model.load_weights('./model_Fold{}.h5'.format(str(i)))
    if i==0:
        pred = model.predict(dtest, verbose=1)
    else:
        pred = pred + model.predict(dtest, verbose=1)

# 後処理
sub_df[label_cols] = pred / CFG['fold']
sub_df.to_csv('submission.csv', index=False)

sub_df.head()